<a href="https://www.kaggle.com/code/fiftythirtyfour/on-time-performance-usa-2023?scriptVersionId=236513020" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/kaggle/input/2023-us-civil-flights-delay-meteo-and-aircraft/US_flights_2023.csv')

In [3]:
df.head().T

,0,1,2,3,4
FlightDate,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06
Day_Of_Week,1,2,3,4,5
Airline,Endeavor Air,Endeavor Air,Endeavor Air,Endeavor Air,Endeavor Air
Tail_Number,N605LR,N605LR,N331PQ,N906XJ,N337PQ
Dep_Airport,BDL,BDL,BDL,BDL,BDL
Dep_CityName,"Hartford, CT","Hartford, CT","Hartford, CT","Hartford, CT","Hartford, CT"
DepTime_label,Morning,Morning,Morning,Morning,Morning
Dep_Delay,-3,-5,-5,-6,-1
Dep_Delay_Tag,0,0,0,0,0
Dep_Delay_Type,Low <5min,Low <5min,Low <5min,Low <5min,Low <5min


In [4]:
df.columns = df.columns.str.lower()
num = [x for x in df.columns if df.dtypes[x] in ('int32', 'int64', 'float32', 'float64')]
cat = [x for x in df.columns if df.dtypes[x] == 'object']
df['otp_dep15'] = np.where(df['dep_delay'] < 15, 1, 0)
target = 'otp_dep15'

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2)

# Data Science

In [6]:
df[num].corrwith(df[target]).sort_values(ascending=False)

aicraft_age          -0.006569
day_of_week          -0.027518
delay_security       -0.034913
flight_duration      -0.042140
delay_weather        -0.099895
delay_nas            -0.182576
delay_carrier        -0.267649
delay_lastaircraft   -0.363774
arr_delay            -0.516317
dep_delay            -0.528165
dep_delay_tag        -0.649433
dtype: float64

In [7]:
df.groupby([target])[['dep_delay']].max()

,dep_delay
otp_dep15,
0,4413
1,14


In [8]:
cat

['flightdate',
 'airline',
 'tail_number',
 'dep_airport',
 'dep_cityname',
 'deptime_label',
 'dep_delay_type',
 'arr_airport',
 'arr_cityname',
 'arr_delay_type',
 'distance_type',
 'manufacturer',
 'model']

In [9]:
df.groupby(['airline'])[target].agg(['mean', 'sum', 'count']).sort_values(by='mean', ascending=False)

,mean,sum,count
airline,,,
Republic Airways,0.892419,255669,286490
Endeavor Air,0.866601,170638,196905
Skywest Airlines Inc.,0.857039,569812,664861
PSA Airlines,0.854071,163189,191072
American Eagle Airlines Inc.,0.843926,189626,224695
Delta Air Lines Inc,0.828925,806487,972931
Alaska Airlines Inc.,0.822735,199631,242643
United Air Lines Inc.,0.800749,576565,720032
Hawaiian Airlines Inc.,0.796235,63573,79842


# Machine

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer


In [11]:
num_pip = Pipeline([
    ('impute', SimpleImputer())
    , ('scaler', StandardScaler())
])

cat_pip = Pipeline([
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

pipe = ColumnTransformer([
    ('num', num_pip, ['arr_delay', 'delay_lastaircraft', 'delay_carrier', 'delay_nas', 'delay_weather', 'delay_security'])
    , ('cat', cat_pip, ['airline'])
])

pipe.fit(train)
train_pre = pipe.transform(train)
test_pre = pipe.transform(test)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [13]:
log = LogisticRegression(max_iter=1000)
log.fit(train_pre, train[target])
cross_val_score(log, train_pre, train[target], scoring='accuracy').mean()

0.9425549744627333

# Notes
It is cheating if I use the delay information of the past. In reality, none of these information are available. How can I create a model without these? 